#### Common evaluation metrics for regression

##### 1)mean absolute error
##### 2)Mean squared value
##### 3) root mean squared error 
##### 4) R squared values

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('linear_reg').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [10]:
data = spark.read.csv('data/Ecommerce_Customers.csv',inferSchema=True, header=True)

In [8]:
Data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [11]:
data.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [13]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [16]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [17]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership']
                            ,outputCol='features')

In [18]:
output = assembler.transform(data)

In [20]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [21]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [22]:
final_data = output.select('features','yearly Amount spent')

In [23]:
final_data.show()

+--------------------+-------------------+
|            features|yearly Amount spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [24]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [25]:
train_data.describe().show()

+-------+-------------------+
|summary|yearly Amount spent|
+-------+-------------------+
|  count|                341|
|   mean|  499.8184013334464|
| stddev|  78.25197569831629|
|    min|  275.9184206503857|
|    max|  765.5184619388373|
+-------+-------------------+



In [27]:
lr = LinearRegression(labelCol='yearly Amount spent')

In [28]:
lr_model = lr.fit(train_data)

In [29]:
test_results = lr_model.evaluate(test_data)

In [31]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-10.176012195728276|
| 11.529370214923233|
|-15.306520496821008|
| -3.495753872985631|
|-2.9726373004245374|
|  6.042578476519452|
|-5.5499333639987185|
|-3.2363215147195206|
|  3.720433634100459|
|  20.09038823469274|
|  4.579165707902007|
|-3.4260657415345577|
|  3.729359099062492|
| -4.283255835180398|
|-1.5543081770168783|
|  17.78129878284227|
|  7.499389399386416|
|-3.9266401561977204|
|-11.146854293243905|
|-17.273212502949775|
+-------------------+
only showing top 20 rows



In [32]:
test_results.rootMeanSquaredError

9.76409213414301

In [34]:
test_results.r2

0.9856575294306313

In [35]:
final_data.describe().show()

+-------+-------------------+
|summary|yearly Amount spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [36]:
unlabeled_data = test_data.select('features')

In [37]:
predictions = lr_model.transform(unlabeled_data)

In [ ]:
predictions.